In [15]:
from requests_html import HTMLSession
import os
import random
import time
session = HTMLSession()

In [16]:
def load_useragents():
    uas = []
    with open("user-agents.txt", 'rb') as uaf:
        for ua in uaf.readlines():
            if ua:
                uas.append(ua.strip()[0:-1-0])
    random.shuffle(uas)
    return uas


def extract_items(indices, r, session):
    for tr in r.html.find('tr'):
        if str(tr.attrs.get('id')).startswith('pair_') is False:
            continue

        name = tr.find('a', first=True).text
        url = list(tr.find('a', first=True).absolute_links)[0]
        id_indice = tr.attrs.get('id').split('pair_')[1]
        
        print('Scraping data', name, id_indice, url)

        headers = {'User-Agent': random.choice(load_useragents())}
        r2 = session.get(url, headers=headers)
        time.sleep(5)
        
        tag = url.split('/')[-1]
        div_info = r2.html.find('.right.general-info', first=True)
        print(div_info)
        type_indice = div_info.find('span')[1].text

        market = div_info.find('span')[3].text.lower()
        market = market.replace(' ', '_').lower()
        country = market
        
        if market == 'global_indices':
            country = 'world'

        div_head = r2.html.find('.instrumentHead', first=True)
        symbol = div_head.find('h1')[0].text.split(' (')[1][:-1]

        current_data = r2.html.find('.current-data', first=True)
        currency = current_data.text.split('Currency in ')[1].split(' ')[0]

        item = {
            'country': country,
            'name': name,
            'full_name': name,
            'tag': tag,
            'id': id_indice,
            'symbol': symbol,
            'currency': currency,
            'class': type_indice,
            'market': market
        }

        indices.append(item)
    
    return indices
    print('Captura finalizada')

In [17]:
indices = [] 

In [ ]:
url = 'https://www.investing.com/indices/world-indices?&majorIndices=on&primarySectors=on&additionalIndices=on&otherIndices=on'
headers = {'User-Agent': random.choice(load_useragents())}
r = session.get(url, headers=headers)
print(r)
indices = extract_items(indices, r, session)
len(indices)

In [ ]:
url = 'https://www.investing.com/indices/global-indices?majorIndices=on&primarySectors=on&bonds=on&additionalIndices=on&otherIndices=on&commodities=on'
headers = {'User-Agent': random.choice(load_useragents())}
r = session.get(url, headers=headers)
print(r)
indices = extract_items(indices, r, session)
len(indices)